In [1]:
import nibabel as nib
import numpy as np
import matplotlib.pyplot as plt

# Load fish movie frames

In [2]:
# load the fish movie
data_path = '../../../local_data/original_files/'

frames = np.loadtxt(data_path + 'frames.csv', delimiter=',')
frames = frames.reshape(frames.shape[0], 120, 200)
print('frames.shape = {}'.format(frames.shape))

frames.shape = (1141, 120, 200)


# Create batches of frames from which features are extracted

For now, each batch has 32 frames to comply with slowfast batches.
<br>
Batch #0: contains 32 frames leading up to frame #0 (in other words, frames[-31:1])
<br>
...
<br>
Batch #31: contains 32 frames leading up to frame #31 (in other words, frames[0:32])

$\implies$ All in all, we have 1141 batches for 1141 frames.

In [3]:
batch_ind = 10
batch_sz = 32
idx = np.arange(batch_ind-batch_sz+1, batch_ind+1, dtype=np.int32)
sample_batched = frames[idx]
print('chosen frame indices: ', idx)
print('sample_batched.shape = {}'.format(sample_batched.shape))

chosen frame indices:  [-21 -20 -19 -18 -17 -16 -15 -14 -13 -12 -11 -10  -9  -8  -7  -6  -5  -4
  -3  -2  -1   0   1   2   3   4   5   6   7   8   9  10]
sample_batched.shape = (32, 120, 200)


# Save frame-batch as a Nifti image

We need an affine matrix that converts the image-coordinates ({i, j, k} centered at the top-left corner of the oldest frame)
<br>
to the scanner-coordinates used by Nifti package ({x, y, z} centered at the bottom-left corner of the oldest frame).
<br>
For an explanation of these coordinates, please refer to https://nipy.org/nibabel/coordinate_systems.html#voxel-coordinates-and-points-in-space
<br>
<img src="file/Nifti affine matrix.png" width="300">

Hopefully, Sift3D also uses Nifti scanner coordinates; therefore, location of keypoints are described in those coordinates.

In [ ]:
affine = np.array([[1, 0, 0, 0],
                   [0, -1, 0, 120],
                   [0, 0, 1, 0],
                   [0, 0, 0, 1]])
sample_nifti = nib.Nifti1Image(sample_batched, affine=affine)
nib.save(sample_nifti, '../../data/sift3d/batch.nii')

# Extract Sift3D keypoints and descriptors from the Nifti image

Sift3D package is downloaded from here:
<br>
https://github.com/bbrister/SIFT3D
<br>
Version 1.4.6 is used in this notebook via command-line interface (CLI).

In [ ]:
# Must install Sift3D package before running below line
!kpSift3D --keys ../../data/sift3d/keys.csv --desc ../../data/sift3d/desc.csv '../../data/sift3d/batch.nii'

# Load descriptros

In [ ]:
desc = np.loadtxt('../../data/sift3d/desc.csv', delimiter=',')
keys = np.loadtxt('../../data/sift3d/keys.csv', delimiter=',')

print(desc.shape)
print(keys.shape)

(484, 771)
(484, 14)


# Extract Sift3D features for all possible batches

In [ ]:
affine = np.array([[1, 0, 0, 0],
                   [0, -1, 0, 120],
                   [0, 0, 1, 0],
                   [0, 0, 0, 1]])

batch_sz = 32
for batch_ind in range(1141):
    idx = np.arange(batch_ind-batch_sz+1, batch_ind+1, dtype=np.int32)
    sample_batched = frames[idx]

    sample_nifti = nib.Nifti1Image(sample_batched, affine=affine)
    nib.save(sample_nifti, '../../data/sift3d/batch.nii')

    !kpSift3D --keys ../../data/sift3d/keys.csv --desc ../../data/sift3d/desc.csv ../../data/sift3d/batch.nii

    desc = np.loadtxt('../../data/sift3d/desc.csv', delimiter=',')
    keys = np.loadtxt('../../data/sift3d/keys.csv', delimiter=',')

    np.savetxt('../../data/sift3d/desc_'+str(batch_ind)+'.csv', desc)
    np.savetxt('../../data/sift3d/keys_'+str(batch_ind)+'.csv', keys)

In [ ]:
# remove temporary files that were created in the above cell
!rm ../../data/sift3d/batch.nii ../../data/sift3d/keys.csv ../../data/sift3d/desc.csv

# Code snippet to load Sift3D keypoints and descriptors

In [ ]:
# load a batch's Sift3D keypoints and descriptions
batch_i = 34
keys = np.loadtxt('../../data/sift3d/keys_'+str(batch_i)+'.csv')
desc = np.loadtxt('../../data/sift3d/desc_'+str(batch_i)+'.csv')

## Descriptor format:
[x, y, z, el0, el1, ..., el767]
<br>
Each row is a single feature descriptor.
<br>
[x, y, z] are the keypoint's coordinates in 3D-image space, and [el0, el1, ..., el767] are the 768 dimensions of the descriptor.

## Keypoint format:
<br>
Each row is a keypoint. The elements of each row are as follows:
<br>
[x, y, z, o, s, $ori_{11}, ori_{12}, ..., ori_{nn}$]
<br>
x - the x-coordinate
<br>
y - the y-coordinate
<br>
z - the z-coordinate
<br>
o - the pyramid octave. To convert to image coordinates, multiply x,y,z by pow(2, o)
<br>
s - the scale coordinate
<br>
$ori_{ij}$ - the ith row, jth column of the orientation matrix